In [ ]:
%load_ext autoreload
%autoreload 2

import pylupnt as pnt
import numpy as np
import matplotlib.pyplot as plt

pnt.Logger.set_level(pnt.Logger.DEBUG)

output_dir = pnt.BASEDIR / "output" / "2025_FeatureMatching"
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
ds = pnt.datasets.Dataset.from_config(
    {
        "inherit_from": "datasets/unreal.yaml",
        "basedir": "/home/shared_ws6/data/unreal_engine/local_traverse/base_1",
        "agent": "rover0",
    }
)

img_data1 = ds[0]["cameras"]["front_left"]
img_data2 = ds[1]["cameras"]["front_left"]

extractor = pnt.FeatureExtractor.from_config({"class": "SuperPoint"})
matcher = pnt.FeatureMatcher.from_config({"class": "SuperGlue"})
feats1 = extractor.extract(img_data1.rgb)
feats2 = extractor.extract(img_data2.rgb)
matches = matcher.match(feats1, feats2)

h, w = img_data1.rgb.shape[:2]
plt.figure(figsize=(10, 4))
plt.imshow(np.hstack([img_data1.rgb, img_data2.rgb]))
plt.scatter(feats1.uv[:, 0], feats1.uv[:, 1], color="cyan", s=2, zorder=10)
plt.scatter(feats2.uv[:, 0] + w, feats2.uv[:, 1], color="magenta", s=2, zorder=10)
plt.axis("off")
plt.show()
plt.show()

plt.figure(figsize=(10, 4))
plt.imshow(np.hstack([img_data1.rgb, img_data2.rgb]))
for i, j in matches.indexes:
    plt.plot(
        [feats1.uv[i, 0], feats2.uv[j, 0] + w],
        [feats1.uv[i, 1], feats2.uv[j, 1]],
        "-",
        color="lime",
        lw=1.0,
        zorder=15,
    )
plt.axis("off")
plt.show()

print(len(feats1), len(feats2), len(matches))

In [ ]:
# Dataset
lusnar_config = {
    "inherit_from": "datasets/lusnar.yaml::lusnar_dataset",
    # "end": 1.0,
    "preload": "none",
    "data": ["rgb_left", "depth", "label"],
}
lusnar_dataset = pnt.datasets.Dataset.from_config(lusnar_config)

In [ ]:
data = lusnar_dataset[0]
img_data = data["cameras"]["front_left"]

# Plot rgb, depth, and label from the data side by side

rgb_img = img_data["rgb"]
depth_img = img_data["depth"]
label_img = img_data["label"]

import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

colors = [plt.get_cmap("tab20")(i) for i in range(len(np.unique(label_img)))]
discrete_cmap = mcolors.ListedColormap(colors)
unique_labels = np.unique(label_img)
label_to_idx = {j: i for i, j in enumerate(unique_labels)}
label_idx_img = np.vectorize(label_to_idx.get)(label_img)

log_min = np.floor(np.nanmin(np.log10(depth_img[depth_img > 0])))
log_max = np.ceil(np.nanmax(np.log10(depth_img[depth_img > 0])))

# Increase figure height for the trajectory so aspect is not forced by 'equal'
fig_width, im_height = 12, 3
fig, axes = plt.subplots(
    1, 4, figsize=(fig_width, im_height), gridspec_kw={"width_ratios": [1, 1, 1, 1]}
)

axes[0].imshow(rgb_img)
axes[0].set_title("RGB")
axes[0].axis("off")

im1 = axes[1].imshow(
    np.log10(np.clip(depth_img, 1e-6, None)), cmap="viridis", vmin=log_min, vmax=log_max
)
axes[1].set_title("Depth")
axes[1].axis("off")
# Move colorbar inside the second (depth) axes using inset_axes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

cax = inset_axes(
    axes[1],
    width="4%",
    height="40%",
    loc="lower left",
    bbox_to_anchor=(0.08, 0.09, 1, 1),
    bbox_transform=axes[1].transAxes,
    borderpad=0,
)
cbar1 = fig.colorbar(im1, cax=cax)
cbar1.ax.tick_params(labelsize="small", length=0, colors="white")
depth_tick_vals = np.arange(log_min, log_max + 1)
cbar1.set_ticks(depth_tick_vals)
cbar1.set_ticklabels([f"$10^{{{int(tick)}}}$" for tick in depth_tick_vals])
cbar1.set_label("Depth [m]", fontsize="small", labelpad=8, color="white")
cbar1.ax.yaxis.label.set_color("white")
for label in cbar1.ax.get_yticklabels():
    label.set_color("white")

im2 = axes[2].imshow(
    label_idx_img, cmap=discrete_cmap, vmin=0, vmax=len(unique_labels) - 1
)
axes[2].set_title("Label")
axes[2].axis("off")

# Add legend using full label names if available
# Use LABEL_NAMES to map unique_labels to their string names if possible
label_names_list = []
for label in unique_labels:
    try:
        label_name = pnt.LABEL_NAMES[label]
    except Exception:
        label_name = str(label)
    label_names_list.append(label_name)

patches = [
    mpatches.Patch(color=colors[i], label=label_name)
    for i, label_name in enumerate(label_names_list)
]

from matplotlib.offsetbox import (
    AnchoredOffsetbox,
    HPacker,
    VPacker,
    TextArea,
    DrawingArea,
)

legend = axes[2].legend(
    handles=patches,
    loc="lower left",
    bbox_to_anchor=(0.01, 0.01, 0.99, 0.99),
    bbox_transform=axes[2].transAxes,
    fontsize="small",
    ncol=1,
    frameon=True,
    # title="Semantic Class",
    title_fontsize="small",
    borderaxespad=0.1,
)

try:
    legend_handles = legend.legend_handles
except AttributeError:
    legend_handles = (
        legend.legendHandles
        if hasattr(legend, "legendHandles")
        else legend.get_patches()
    )

for lh in legend_handles:
    lh.set_alpha(1)
    lh.set_linewidth(1)

axes[3].plot(-lusnar_dataset.positions[:, 1], lusnar_dataset.positions[:, 0])
axes[3].set_xlabel("X [m]")
axes[3].set_ylabel("Y [m]")
axes[3].set_title("Trajectory")
axes[3].grid(True)
# Remove axes[3].axis("equal") so the plot fills the subplot, making height match others

plt.tight_layout(rect=[0, 0, 1, 1])
plt.savefig(output_dir / "lusnar_rgb_depth_label.pdf", dpi=300)
plt.show()

In [ ]:
unreal_config = "datasets/unreal.yaml"
unreal_dataset = pnt.datasets.Dataset.from_config(unreal_config)

In [ ]:
from pylupnt.features import SuperPoint

extractor_configs = {
    "SIFT": {"class": "Sift"},
    "ORB": {"class": "Orb"},
    "AKAZE": {"class": "Akaze"},
    "BRISK": {"class": "Brisk"},
    "SuperPoint": {"class": "SuperPoint"},
}

matcher_configs = {
    "Flann": {"class": "FlannMatcher"},
    "BruteForce": {"class": "BruteForceMatcher"},
    "SuperGlue": {"class": "SuperGlue"},
    "LightGlue": {"class": "LightGlue"},
}

extractors = {
    k: pnt.FeatureExtractor.from_config(v) for k, v in extractor_configs.items()
}
matchers = {k: pnt.FeatureMatcher.from_config(v) for k, v in matcher_configs.items()}

In [ ]:
def plot_features(feats1, feats2, img_data1, img_data2, matches, filename):
    h, w = img_data1.rgb.shape[:2]

    # Check valid feature coordinates in image 1
    u1 = feats1.uv[:, 0].astype(np.int32).clip(0, w - 1)
    v1 = feats1.uv[:, 1].astype(np.int32).clip(0, h - 1)

    depth1 = img_data1.depth[v1, u1]

    # Convert to 3D world coordinates
    xyz_c1 = pnt.uv_to_xyz(feats1.uv, depth1, img_data1.intrinsics)
    xyz1_w = pnt.apply_transform(img_data1.world_T_cam, xyz_c1)

    # Project to image 2
    uv2_true, depth2 = pnt.xyz_to_uv(
        xyz1_w, img_data2.intrinsics, img_data2.world_T_cam, return_depth=True
    )

    # Filter valid projections
    valid_mask2: np.ndarray = (
        (depth1 > 0)
        & (depth2 > 0)
        & (0 <= uv2_true[:, 0])
        & (uv2_true[:, 0] < w)
        & (0 <= uv2_true[:, 1])
        & (uv2_true[:, 1] < h)
    )

    # Compute pairwise distances between uv2_true and feats2.uv
    all_dists = np.linalg.norm(uv2_true[:, None, :] - feats2.uv[None, :, :], axis=2)
    dist_true2feats = np.min(all_dists, axis=1)
    dist_feats2true = np.min(all_dists, axis=0)

    # Match distance
    gt_thresh = 0.01 * w
    dist_match = dist_feats2true[matches.indexes[:, 1]]
    dists_normed = np.clip(dist_match / gt_thresh / 2.0, 0, 1)
    colors = np.vstack([dists_normed, 1 - dists_normed, np.zeros(len(dists_normed))]).T

    # Precision and recall
    matched = np.zeros(len(feats1), dtype=np.bool)
    matched[matches.indexes[:, 0]] = True
    pos = (dist_true2feats < gt_thresh) & valid_mask2
    neg = (dist_true2feats >= gt_thresh) | ~valid_mask2
    tp = matched & pos
    fp = matched & neg
    fn = ~matched & pos
    prec = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    rec = np.sum(tp) / (np.sum(tp) + np.sum(fn))

    print(f"Positive: {np.sum(pos)}, Negative: {np.sum(neg)}")
    print(f"TP: {np.sum(tp)}, FP: {np.sum(fp)}, FN: {np.sum(fn)}")
    print(f"Precision: {prec:.2f}, Recall: {rec:.2f}")

    # Subsample matches
    n_matches = len(matches.indexes)
    idxs_match = np.argsort(matches.distances)[:n_matches]
    idxs_uv1, idxs_uv2 = matches.indexes[idxs_match].T

    pnp_config = {"threshold": 1.0, "confidence": 0.999, "max_iterations": 1000}
    pnp_solver = pnt.PnpSolver(pnp_config)
    K = pnt.make_camera_matrix(img_data2.intrinsics)
    pnp_result = pnp_solver.solve(
        xyz_c1[matches.indexes[:, 0]], feats2.uv[matches.indexes[:, 1]], K
    )

    # Compute error
    c2Tc1_pnp = pnp_result.tgt_T_src
    wTc1 = img_data1.world_T_cam
    c2Tw = pnt.invert_transform(img_data2.world_T_cam)
    c2Tc1_gt = c2Tw @ wTc1

    ea_t_pnp = np.linalg.norm(c2Tc1_gt[:3, 3] - c2Tc1_pnp[:3, 3])
    ea_R_pnp = pnt.rotation_angle(c2Tc1_pnp[:3, :3] @ c2Tc1_gt[:3, :3].T) * pnt.DEG

    print(f"Translation error: {ea_t_pnp:.2f} m")
    print(f"Rotation error: {ea_R_pnp:.2f} deg")

    h, w = img_data1["rgb"].shape[:2]
    ws = 10
    sep = np.ones((h, ws, 3))
    plt.figure(figsize=(10, 4))
    plt.imshow(np.hstack([img_data1["rgb"], sep, img_data2["rgb"]]))
    plt.scatter(feats1.uv[:, 0], feats1.uv[:, 1], color="cyan", s=2, zorder=10)
    plt.scatter(
        feats2.uv[:, 0] + w + ws, feats2.uv[:, 1], color="magenta", s=2, zorder=10
    )
    plt.scatter([], [], color="cyan", s=10, label="Extracted Features 1")
    plt.scatter([], [], color="magenta", s=10, label="Extracted Features 2")
    plt.axis("off")
    plt.xlim(0, 2 * w + ws)
    plt.ylim(h, 0)
    plt.legend(loc="upper right")
    plt.savefig(output_dir / f"{filename}_all.pdf", dpi=300)
    plt.show()

    plt.figure(figsize=(10, 4))
    plt.imshow(np.hstack([img_data1["rgb"], sep, img_data2["rgb"]]))
    plt.scatter(feats1.uv[fp, 0], feats1.uv[fp, 1], color="red", s=2, zorder=10)
    plt.scatter(feats1.uv[fn, 0], feats1.uv[fn, 1], color="yellow", s=2, zorder=10)
    plt.scatter(feats1.uv[tp, 0], feats1.uv[tp, 1], color="lime", s=2, zorder=10)
    plt.scatter(
        uv2_true[tp & valid_mask2, 0] + w + ws,
        uv2_true[tp & valid_mask2, 1],
        color="lime",
        s=2,
        zorder=12,
    )
    plt.scatter(
        uv2_true[fp & valid_mask2, 0] + w + ws,
        uv2_true[fp & valid_mask2, 1],
        color="red",
        s=2,
        zorder=12,
    )
    plt.scatter(
        uv2_true[fn & valid_mask2, 0] + w + ws,
        uv2_true[fn & valid_mask2, 1],
        color="yellow",
        s=2,
        zorder=10,
    )
    plt.scatter([], [], color="lime", s=10, label=f"Correct ({np.sum(tp)} TP)")
    plt.scatter([], [], color="red", s=10, label=f"Incorrect ({np.sum(fp)} FP)")
    plt.scatter([], [], color="yellow", s=10, label=f"Missed ({np.sum(fn)} FN)")
    plt.scatter([], [], s=0.001, label=f"Precision: {prec:.2f}, Recall: {rec:.2f}")
    plt.axis("off")
    plt.xlim(0, 2 * w + ws)
    plt.ylim(h, 0)
    plt.legend(loc="upper right")
    plt.savefig(output_dir / f"{filename}_fp_fn.pdf", dpi=300)
    plt.show()

    plt.figure(figsize=(10, 4))
    plt.imshow(np.hstack([img_data1["rgb"], sep, img_data2["rgb"]]))
    # plt.scatter(feats1.uv[idxs_uv1, 0], feats1.uv[idxs_uv1, 1], color="cyan", s=2, zorder=10)
    # plt.scatter(feats2.uv[idxs_uv2, 0] + w + ws, feats2.uv[idxs_uv2, 1], color="cyan", s=2, zorder=10)
    for k, (i, j) in enumerate(zip(idxs_uv1, idxs_uv2)):
        pt1, pt2 = feats1.uv[i], feats2.uv[j]
        plt.plot(
            [pt1[0], pt2[0] + w + ws],
            [pt1[1], pt2[1]],
            "-",
            color=colors[idxs_match[k]],
            lw=1.0,
            zorder=15,
        )
    plt.axis("off")
    plt.plot([], [], color="lime", lw=2.0, label="Low Error")
    plt.plot([], [], color="red", lw=2.0, label="High Error")
    plt.xlim(0, 2 * w + ws)
    plt.ylim(h, 0)
    plt.legend(loc="upper right")
    plt.savefig(output_dir / f"{filename}_matches.pdf", dpi=300)
    plt.show()

In [ ]:
plt.imshow(unreal_dataset[180]["cameras"]["front_left"]["rgb"])
plt.show()

In [ ]:
cam_name = "front_left"
img_data1 = unreal_dataset[90]["cameras"][cam_name]
img_data2 = unreal_dataset[94]["cameras"][cam_name]

feats1 = extractors["SuperPoint"].extract(img_data1.rgb)
feats2 = extractors["SuperPoint"].extract(img_data2.rgb)

feats1 = filter_features_by_depth(feats1, img_data1)
feats2 = filter_features_by_depth(feats2, img_data2)

matches = matchers["SuperGlue"].match(feats1, feats2)

filename = "extracted_features_consecutive"
plot_features(feats1, feats2, img_data1, img_data2, matches, filename)

In [ ]:
img_data1 = unreal_dataset[90]["cameras"]["front"]
img_data2 = unreal_dataset[155]["cameras"]["front"]

feats1 = extractors["SuperPoint"].extract(img_data1.rgb)
feats2 = extractors["SuperPoint"].extract(img_data2.rgb)

feats1 = filter_features_by_depth(feats1, img_data1)
feats2 = filter_features_by_depth(feats2, img_data2)

matches = matchers["SuperGlue"].match(feats1, feats2)

filename = "extracted_features_nonconsecutive"
plot_features(feats1, feats2, img_data1, img_data2, matches, filename)

In [ ]:
end = 10
lusnar_config = pnt.Config(
    {"inherit_from": "datasets/lusnar.yaml", "end": end, "preload": "cpu"}
)

lusnar_dataset = pnt.datasets.Dataset.from_config(lusnar_config)

In [ ]:
from tqdm import tqdm

pnp_vo_config = pnt.Config(
    {
        "feature_extractor": {"class": "Orb"},
        "feature_matcher": {"class": "BruteForceMatcher"},
    }
)
pnp_vo = pnt.PnpVo(pnp_vo_config)

pnt.Logger.set_level(pnt.Logger.INFO)

idxs_match = range(len(lusnar_dataset))
for i in tqdm(idxs_match):
    data = lusnar_dataset[i]["cameras"]["front_left"]

    img_data = pnt.ImageData(data)
    # img_data.rgb = data["rgb"]
    # img_data.depth = data["depth"]
    # img_data.intrinsics = pnt.CameraIntrinsics(data["intrinsics"])
    # img_data.body_T_cam = data["body_T_cam"]

    success = pnp_vo.process_mono(img_data)
    if not success:
        break

In [ ]:
fig = go.Figure()
wTb0 = lusnar_dataset[0]["world_T_body"]
poses_gt = [lusnar_dataset[i]["world_T_body"] for i in idxs_match]
# poses_gt = [lusnar_dataset[i]["world_T_body"] for i in range(85,100)]
poses_pnp = []
for wTb in pnp_vo.get_poses():
    poses_pnp.append(wTb0 @ wTb)

pnt.plot_poses(poses_pnp, fig=fig, width=5, color="red")
pnt.plot_poses(poses_gt, fig=fig, width=10, color="blue")
fig.update_layout(scene_aspectmode="data", width=800, height=400)
fig.show()